In [1]:
import random
import gym
import numpy as np
import pandas as pd
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K

import tensorflow as tf

rg/install/errors

for some common reasons and solutions.  Include the entire stack trace
above this error message when asking for help.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\ivukotic\AppData\Roaming\Python\Python36\site-packages\IPython\core\interactiveshell.py", line 2018, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ImportError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Program Files\Python36\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Program Files\Python36\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Program Files\Python36\lib\sit

TypeError: must be str, not list

In [2]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    """Huber loss for Q Learning
    References: https://en.wikipedia.org/wiki/Huber_loss
                https://www.tensorflow.org/api_docs/python/tf/losses/huber_loss
    """

    def _huber_loss(self, y_true, y_pred, clip_delta=1.0):
        error = y_true - y_pred
        cond = K.abs(error) <= clip_delta

        squared_loss = 0.5 * K.square(error)
        quadratic_loss = 0.5 * K.square(clip_delta) + clip_delta * (K.abs(error) - clip_delta)

        return K.mean(tf.where(cond, squared_loss, quadratic_loss))

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss=self._huber_loss, optimizer=Adam(lr=self.learning_rate))
        return model

    def update_target_model(self):
        # copy weights from model to target_model
        self.target_model.set_weights(self.model.get_weights())

    def memorize(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
            if done:
                target[0][action] = reward
            else:
                # a = self.model.predict(next_state)[0]
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(t)
                # target[0][action] = reward + self.gamma * t[np.argmax(a)]
            self.model.fit(state, target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


In [3]:
env = gym.make('gym_cache:Cache-v0')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
done = False

keys in file: data/ANALY_MWT2_UCORE_processed.h5 : ['/data']
                 1   2     3    4     5  6       kB      fID
time                                                        
1.575759e+09  4065  58  4727  587  2598  0  4015645  1490050
1.575759e+09  4065  58  4727  587  2598  0  4021837  1490618
1.575759e+09  4065  58  4727  587  2598  0  4023917  1490799
1.575759e+09  4065  58  4727  587  2598  0  3998864  1488604
1.575759e+09  4065  58  4727  587  2598  0  4072178  1494759


W0314 22:07:05.966164 139812251588352 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0314 22:07:05.982715 139812251588352 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0314 22:07:05.985616 139812251588352 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0314 22:07:06.028197 139812251588352 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0314 22:07:06.038518 139812251588352 deprecation.py:323

environment loaded!
cache size [kB]: 21474836480


In [4]:
EPISODES=2
batch_size = 10000
# accesses=100000
accesses=3090162

for e in range(EPISODES):
#     print('episode:',e)
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(accesses):
        if not time%10000:
            print('access:', time)
        # env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)

        next_state = np.reshape(next_state, [1, state_size])
        agent.memorize(state, action, reward, next_state, done)
        state = next_state
        if done:
            print('done time:',time)
            agent.update_target_model()
            print("episode: {}/{}, score: {}, e: {:.2}".format(e, EPISODES, time, agent.epsilon))
            break
#         if len(agent.memory) > batch_size:
#             agent.replay(batch_size)

access: 0
access: 10000
cache cleanup starting on access: 16679
cleaned 799 files.
cache cleanup starting on access: 16887
cleaned 506 files.
cache cleanup starting on access: 17097
cleaned 322 files.
cache cleanup starting on access: 17309
cleaned 287 files.
cache cleanup starting on access: 17518
cleaned 889 files.
cache cleanup starting on access: 17968
cleaned 551 files.
cache cleanup starting on access: 18178
cleaned 744 files.
cache cleanup starting on access: 18388
cleaned 3057 files.
cache cleanup starting on access: 19125
cleaned 776 files.
cache cleanup starting on access: 19342
cleaned 617 files.
access: 20000
cache cleanup starting on access: 20142
cleaned 2358 files.
cache cleanup starting on access: 20367
cleaned 914 files.
cache cleanup starting on access: 21028
cleaned 406 files.
cache cleanup starting on access: 21367
cleaned 672 files.
cache cleanup starting on access: 22430
cleaned 301 files.
cache cleanup starting on access: 24489
cleaned 243 files.
cache cleanup st

KeyboardInterrupt: 